In [1]:
%reload_ext autoreload
%autoreload 2

In [11]:
from parsers import arbitrary, text_utils
import artm
import hierarchy_utils
import pickle
from spectrum import arrange_topics
#import arranging.api as api
from crossmin import CrossMinimizer
import numpy as np

In [2]:


T = lambda lid, tid: "level_%d_%s" % (lid, tid)

def from_artm_tid(artm_tid):
    # This is due to hARTM bug
    if artm_tid.startswith("level_0_"):
        return (0, artm_tid[8:])
    else:
        lid, tid = artm_tid[5:].split("_", 1)
        lid = int(lid)
        return (lid, tid)

MODEL_PATH = "hartm/"

artm_model = hierarchy_utils.hARTM(theta_columns_naming="title",
                                   cache_theta=True)
artm_model.load(MODEL_PATH)


/Users/aksholokhov/.anaconda3/lib/python3.6/site-packages/bigartm-0.8.3-py3.6.egg/artm/master_component.py:604: DeprecationWarning: invalid escape sequence \*
/Users/aksholokhov/.anaconda3/lib/python3.6/site-packages/bigartm-0.8.3-py3.6.egg/artm/master_component.py:714: DeprecationWarning: invalid escape sequence \d
/Users/aksholokhov/.anaconda3/lib/python3.6/site-packages/bigartm-0.8.3-py3.6.egg/artm/master_component.py:783: DeprecationWarning: 'async' and 'await' will become reserved keywords in Python 3.7


NameError: name 'pickle' is not defined

In [8]:
artm_extra_info = pickle.load(open(MODEL_PATH + "/extra_info.dump", "rb"))

In [9]:
# Extract Phi, Psi and Theta matrices
phis = []
psis = []
#theta = artm_extra_info["theta"]
# theta = pickle.load(open(THETA_MODEL_PATH, "rb"))["theta"]
for level_idx, artm_level in enumerate(artm_model._levels):
    phis.append(artm_level.get_phi(class_ids="flat_tag"))
    if level_idx > 0:
        psis.append(artm_level.get_psi())
        
phi0_topic_titles = list(filter(lambda x: x.startswith("topic"), phis[0].columns))
phi1_topic_titles = list(filter(lambda x: x.startswith("topic"), phis[1].columns))

In [12]:
new_phi1_topic_order = np.array(phi1_topic_titles)[arrange_topics(phis[1][phi1_topic_titles].values)]
phis[1] = phis[1][new_phi1_topic_order]
psis[0] = psis[0].loc[new_phi1_topic_order]

In [13]:
children_threshold = 0.05

D = np.zeros((len(phi0_topic_titles), len(phi1_topic_titles)))

for parent_id, parent in enumerate(phi0_topic_titles):
    for child_id, maybe_child in enumerate(phi1_topic_titles):
        if psis[0].loc[maybe_child, parent] > children_threshold:
            D[parent_id, child_id] = 1

In [14]:
cm = CrossMinimizer(D)
idx = cm.solve(mode="auto", model=None)
new_phi0_topic_order = np.array(phi1_topic_titles)[idx]

/Users/aksholokhov/.anaconda3/lib/python3.6/site-packages/pulp/solvers.py:71: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
  'os':operating_system, 'arch':arch})


In [15]:
new_phi0_topic_order

array(['topic_3', 'topic_1', 'topic_7', 'topic_9', 'topic_10', 'topic_12',
       'topic_2', 'topic_4', 'topic_13', 'topic_11', 'topic_17',
       'topic_16', 'topic_6', 'topic_14', 'topic_18', 'topic_5', 'topic_0',
       'topic_15', 'topic_8'], 
      dtype='<U8')

In [16]:
new_phi1_topic_order

array(['topic_58', 'topic_6', 'topic_19', 'topic_36', 'topic_3',
       'topic_59', 'topic_30', 'topic_38', 'topic_5', 'topic_49',
       'topic_0', 'topic_21', 'topic_50', 'topic_55', 'topic_16',
       'topic_2', 'topic_46', 'topic_20', 'topic_56', 'topic_52',
       'topic_24', 'topic_53', 'topic_4', 'topic_22', 'topic_34',
       'topic_26', 'topic_33', 'topic_42', 'topic_51', 'topic_45',
       'topic_18', 'topic_28', 'topic_37', 'topic_40', 'topic_12',
       'topic_31', 'topic_7', 'topic_10', 'topic_14', 'topic_1',
       'topic_48', 'topic_17', 'topic_23', 'topic_39', 'topic_43',
       'topic_8', 'topic_9', 'topic_41', 'topic_44', 'topic_13',
       'topic_29', 'topic_15', 'topic_54', 'topic_35', 'topic_32',
       'topic_11', 'topic_27', 'topic_57', 'topic_25', 'topic_47'], 
      dtype='<U8')

In [17]:
def get_topic_name(lid, tid):
    return ", ".join(phis[lid][tid].sort_values()[-3:][::-1].index)

In [18]:
print("\n".join(map(lambda t: t + " \t" + get_topic_name(0, t), new_phi0_topic_order)))

topic_3 	культура, литература, культурология
topic_1 	медицина, эволюция, антропология
topic_7 	социология, социология_повседневности, дюркгейм_эмиль
topic_9 	психология, люди_науки, история_науки
topic_10 	палеонтология, биохимия, стволовые_клетки
topic_12 	наука, управление_проектами, работа
topic_2 	информационная_безопасность, копирайт, будущее
topic_4 	история, история_россии, ссср
topic_13 	химия, нанотехнологии, материаловедение
topic_11 	физика, физика_элементарных_частиц, квантовая_физика
topic_17 	экология, зоология, этология
topic_16 	средневековье, мифология, биоинформатика
topic_6 	лингвистика, язык, право
topic_14 	образование, университет, школа
topic_18 	мозг, нейробиология, искусственный_интеллект
topic_5 	астрономия, астрофизика, вселенная
topic_0 	философия, россия, география
topic_15 	математика, информационные_технологии, технологии
topic_8 	общество, экономика, россия


In [19]:
print("\n".join(map(lambda t: t + " \t" + get_topic_name(0, t), phi0_topic_titles)))

topic_0 	философия, россия, география
topic_1 	медицина, эволюция, антропология
topic_2 	информационная_безопасность, копирайт, будущее
topic_3 	культура, литература, культурология
topic_4 	история, история_россии, ссср
topic_5 	астрономия, астрофизика, вселенная
topic_6 	лингвистика, язык, право
topic_7 	социология, социология_повседневности, дюркгейм_эмиль
topic_8 	общество, экономика, россия
topic_9 	психология, люди_науки, история_науки
topic_10 	палеонтология, биохимия, стволовые_клетки
topic_11 	физика, физика_элементарных_частиц, квантовая_физика
topic_12 	наука, управление_проектами, работа
topic_13 	химия, нанотехнологии, материаловедение
topic_14 	образование, университет, школа
topic_15 	математика, информационные_технологии, технологии
topic_16 	средневековье, мифология, биоинформатика
topic_17 	экология, зоология, этология
topic_18 	мозг, нейробиология, искусственный_интеллект


In [20]:
phi0_flat_spectrum = np.array(phi0_topic_titles)[arrange_topics(phis[0][phi0_topic_titles].values)]

In [21]:
print("\n".join(map(lambda t: t + " \t" + get_topic_name(0, t), phi0_flat_spectrum)))

topic_13 	химия, нанотехнологии, материаловедение
topic_5 	астрономия, астрофизика, вселенная
topic_11 	физика, физика_элементарных_частиц, квантовая_физика
topic_15 	математика, информационные_технологии, технологии
topic_10 	палеонтология, биохимия, стволовые_клетки
topic_2 	информационная_безопасность, копирайт, будущее
topic_16 	средневековье, мифология, биоинформатика
topic_4 	история, история_россии, ссср
topic_8 	общество, экономика, россия
topic_0 	философия, россия, география
topic_3 	культура, литература, культурология
topic_6 	лингвистика, язык, право
topic_7 	социология, социология_повседневности, дюркгейм_эмиль
topic_12 	наука, управление_проектами, работа
topic_14 	образование, университет, школа
topic_9 	психология, люди_науки, история_науки
topic_18 	мозг, нейробиология, искусственный_интеллект
topic_17 	экология, зоология, этология
topic_1 	медицина, эволюция, антропология


In [22]:
# Двухуровневый спектр
pickle.dump((new_phi0_topic_order, new_phi1_topic_order, D), open("spectrum.dump", "wb"))

In [23]:
# Одноуровневый спектр
level_0_topics_ids = list(map(lambda tid: "level_0_%s" % tid, phi0_flat_spectrum))
pickle.dump(level_0_topics_ids, open("flat_spectrum.dump", "wb"))